In [155]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup  #HTML 파싱
from urllib.parse import quote  #검색어 인코딩
import time  #슬립 사용
import requests  #HTML 가져오기
import datetime

In [156]:
jan_vod = pd.read_csv('./LG_data/00. 원본/vod_20240101.csv')
feb_vod = pd.read_csv('./LG_data/00. 원본/vod_20240201.csv')
mar_vod = pd.read_csv('./LG_data/00. 원본/vod_20240301.csv')

jan_vod.info()
feb_vod.info()
mar_vod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7007 entries, 0 to 7006
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           7007 non-null   int64 
 1   asset_nm        7007 non-null   object
 2   ct_cl           7007 non-null   object
 3   genre_of_ct_cl  7007 non-null   object
 4   use_tms         7007 non-null   int64 
 5   SMRY            7004 non-null   object
 6   ACTR_DISP       7004 non-null   object
 7   disp_rtm        7007 non-null   object
 8   strt_dt         7007 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 492.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7770 entries, 0 to 7769
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           7770 non-null   int64 
 1   asset_nm        7770 non-null   object
 2   ct_cl           7770 non-null   object
 3   genre_of_ct_cl  7770 non-nu

In [157]:
# jan_to_mar_vod = pd.concat([jan_vod, feb_vod, mar_vod])
# jan_to_mar_vod.info() #22557
# concat : index가 중복되는 문제

# 1) df. reset_index(drop=True, inplace = True)
# : reset_index 특성상 기존의 인덱스는 feature로 생성되고
# 새로운 인덱스가 sort되어서 할당 > drop = True 옵션 작성 필요

# 2) df = pd.concat([df1, df2], ignore_index = True)

# Sol---------------------------------------------------------------
#1) 
# jan_to_mar_vod.reset_index(drop = True, inplace = True)
# jan_to_mar_vod

#2) 
jan_to_mar_vod = pd.concat([jan_vod, feb_vod, mar_vod], ignore_index= True)
jan_to_mar_vod


,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,66875000,장군재상 10회(A),TV드라마,외화 시리즈,2472,조옥근은 엽소에게 호청의 짝사랑을 전하지만 엽소는 오히려 호청을 찾아가 호되게 나무...,"마사순,성일륜,정천,왕초연,장준명",00:43,20240123085102
1,66705000,수호지의 무송 04회.,TV드라마,외화 시리즈,1891,무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...,"유대경,번장강,손요기,장한",00:45,20240116203741
2,66705000,수호지의 무송 04회.,TV드라마,외화 시리즈,632,무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...,"유대경,번장강,손요기,장한",00:45,20240117193153
3,66437000,콩순이 율동교실 4기 01회,키즈,기타,60,콩순이 아이스크림 좋아송. 아이스크림 너무 좋아~! 콩순이와 친구들이 함께 부르는 ...,콩순이,00:01,20240113184221
4,66841000,알사탕,키즈,학습,480,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,00:08,20240113074350
...,...,...,...,...,...,...,...,...,...
22552,66556000,괴물(2023)(예고),영화,드라마,5,싱글맘 사오리는 아들 미나토의 행동에서 이상 기운을 감지한다. 용기를 내 찾아간 학...,"안도 사쿠라,다나카 유코,나가야마 에이타,쿠로카와 소야,히이라기 히나타",00:01,20240314131734
22553,63086000,괴물(2023)(예고),영화,드라마,60,싱글맘 사오리는 아들 미나토의 행동에서 이상 기운을 감지한다. 용기를 내 찾아간 학...,"안도 사쿠라,다나카 유코,나가야마 에이타,쿠로카와 소야,히이라기 히나타",00:01,20240310190508
22554,63017000,만난지3분만에합체 저랑섹스한번하실래요(예고),영화,성인,60,일본 최고의 색녀 시노다 아유미는 마음대로 골라 먹는다. 길거리를 돌아다니며 즉석에...,시노다 아유미,00:01,20240330103945
22555,66056000,만난지3분만에합체 저랑섹스한번하실래요(예고),영화,성인,60,일본 최고의 색녀 시노다 아유미는 마음대로 골라 먹는다. 길거리를 돌아다니며 즉석에...,시노다 아유미,00:01,20240330171645


성인, 키즈 컨텐츠 삭제 / 영화-기타 중 성인 컨텐츠 삭제

In [158]:
#df[(조건) | (조건)].index
filtered_vod = jan_to_mar_vod.drop(jan_to_mar_vod[(jan_to_mar_vod['ct_cl'] == '키즈') | (jan_to_mar_vod['genre_of_ct_cl'] == '성인')
                                                  | ((jan_to_mar_vod['ct_cl'] == '기타') & (jan_to_mar_vod['genre_of_ct_cl'] == '기타'))
                                                  | ((jan_to_mar_vod['ct_cl'] == '영화') & (jan_to_mar_vod['genre_of_ct_cl'] == '기타'))].index, 
                    axis= 0)

filtered_vod.columns = ['ID', 'title', 'category', 'genre_category', 'viewing_time', 'SMRY', 'cast', 'running_time', 'viewing_start_time']

filtered_vod.head()

#성인, 키즈 컨텐츠 삭제
filtered_vod.info() #17181

#SMRY, ACTR_DISP 에서 결측치 4개 확인
print(filtered_vod['SMRY'][filtered_vod['SMRY'].isna()])
print(filtered_vod['cast'][filtered_vod['cast'].isna()])
print(filtered_vod.loc[4257:4260])
#극장판 주술회전0, 악의 연대기, 겨울 왕국(더빙), 가족끼리 왜 이래 23회

#결측치 삭제
filtered_vod.dropna(axis= 0, inplace= True)
print(filtered_vod.info()) #16965

<class 'pandas.core.frame.DataFrame'>
Index: 16968 entries, 0 to 22556
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  16968 non-null  int64 
 1   title               16968 non-null  object
 2   category            16968 non-null  object
 3   genre_category      16968 non-null  object
 4   viewing_time        16968 non-null  int64 
 5   SMRY                16965 non-null  object
 6   cast                16965 non-null  object
 7   running_time        16968 non-null  object
 8   viewing_start_time  16968 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 1.3+ MB
4257    NaN
4258    NaN
4259    NaN
Name: SMRY, dtype: object
4257    NaN
4258    NaN
4259    NaN
Name: cast, dtype: object
            ID               title category genre_category  viewing_time  \
4257  65386000      (자막)극장판 주술회전 0       영화        액션/어드벤쳐          1606   
4258  61395000    (평생소장)(HD)악의 연대기      미분류

In [159]:
filtered_vod.head()

,ID,title,category,genre_category,viewing_time,SMRY,cast,running_time,viewing_start_time
0,66875000,장군재상 10회(A),TV드라마,외화 시리즈,2472,조옥근은 엽소에게 호청의 짝사랑을 전하지만 엽소는 오히려 호청을 찾아가 호되게 나무...,"마사순,성일륜,정천,왕초연,장준명",00:43,20240123085102
1,66705000,수호지의 무송 04회.,TV드라마,외화 시리즈,1891,무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...,"유대경,번장강,손요기,장한",00:45,20240116203741
2,66705000,수호지의 무송 04회.,TV드라마,외화 시리즈,632,무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...,"유대경,번장강,손요기,장한",00:45,20240117193153
7,66875000,가남전 03회(A),TV드라마,외화 시리즈,636,"황궁에서 다시 만난 이겸과 강보녕, 조소는 드디어 서로의 신분을 알게 되고, 이겸과...","쥐징이,쩡순시,왕줘청,왕이팅",00:47,20240114234823
8,66875000,가남전 03회(A),TV드라마,외화 시리즈,1789,"황궁에서 다시 만난 이겸과 강보녕, 조소는 드디어 서로의 신분을 알게 되고, 이겸과...","쥐징이,쩡순시,왕줘청,왕이팅",00:47,20240115172619


In [160]:
time = pd.to_datetime('0:00:41', format = 'mixed')
print(time)
time.to_datetime64()

2024-04-24 00:00:41


numpy.datetime64('2024-04-24T00:00:41.000000000')

시계열 데이터

- 시청 시각(viewing_time)

In [161]:
time = filtered_vod['viewing_time'].values
# print(time)
# # dir(time)

viewing_time = []
for sec in time :
  second = str(sec%60) #초에서 60으로 나눈 나머지
  hour = str(sec//60//60) #분을 시간으로 환산한 몫
  minute = (sec//60)%60 #환산된 분을 60으로 나눈 나머지
  if minute < 10 :
    minute = ''.join(['0', str(minute)])
  else :
    minute = minute
  time = ':'.join([hour, str(minute), second])
  viewing_time.append(time)

viewing_data = []
for value in viewing_time :
  time_data = datetime.datetime.strptime(value, '%H:%M:%S').time()
  viewing_data.append(time_data)
  
filtered_vod['viewing_time'] = viewing_data

# print(filtered_vod['viewing_time'])

# filtered_vod['viewing_time'] = pd.to_datetime(viewing_time, format='mixed')
# viewing_time = filtered_vod['viewing_time'].dt.time
# filtered_vod['viewing_time'] = viewing_time
# # print(filtered_vod['viewing_time'])


print(filtered_vod.info())
print(filtered_vod.head())

<class 'pandas.core.frame.DataFrame'>
Index: 16965 entries, 0 to 22556
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  16965 non-null  int64 
 1   title               16965 non-null  object
 2   category            16965 non-null  object
 3   genre_category      16965 non-null  object
 4   viewing_time        16965 non-null  object
 5   SMRY                16965 non-null  object
 6   cast                16965 non-null  object
 7   running_time        16965 non-null  object
 8   viewing_start_time  16965 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 1.3+ MB
None
         ID         title category genre_category viewing_time  \
0  66875000   장군재상 10회(A)    TV드라마         외화 시리즈     00:41:12   
1  66705000  수호지의 무송 04회.    TV드라마         외화 시리즈     00:31:31   
2  66705000  수호지의 무송 04회.    TV드라마         외화 시리즈     00:10:32   
7  66875000    가남전 03회(A)    TV드라마         외화 시리즈 

- 방영 시간(영화 시간, 시리즈 시간) - running_time

In [162]:
running_data = []
for value in filtered_vod['running_time'] :
  time_data = datetime.datetime.strptime(value, '%H:%M').time()
  running_data.append(time_data)
  
filtered_vod['running_time'] = running_data

# running_time = pd.to_datetime(filtered_vod['running_time'], format= 'mixed')
# running_time = running_time.dt.time
# filtered_vod['running_time'] = running_time

# filtered_vod.info()


In [163]:
filtered_vod.head()


,ID,title,category,genre_category,viewing_time,SMRY,cast,running_time,viewing_start_time
0,66875000,장군재상 10회(A),TV드라마,외화 시리즈,00:41:12,조옥근은 엽소에게 호청의 짝사랑을 전하지만 엽소는 오히려 호청을 찾아가 호되게 나무...,"마사순,성일륜,정천,왕초연,장준명",00:43:00,20240123085102
1,66705000,수호지의 무송 04회.,TV드라마,외화 시리즈,00:31:31,무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...,"유대경,번장강,손요기,장한",00:45:00,20240116203741
2,66705000,수호지의 무송 04회.,TV드라마,외화 시리즈,00:10:32,무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...,"유대경,번장강,손요기,장한",00:45:00,20240117193153
7,66875000,가남전 03회(A),TV드라마,외화 시리즈,00:10:36,"황궁에서 다시 만난 이겸과 강보녕, 조소는 드디어 서로의 신분을 알게 되고, 이겸과...","쥐징이,쩡순시,왕줘청,왕이팅",00:47:00,20240114234823
8,66875000,가남전 03회(A),TV드라마,외화 시리즈,00:29:49,"황궁에서 다시 만난 이겸과 강보녕, 조소는 드디어 서로의 신분을 알게 되고, 이겸과...","쥐징이,쩡순시,왕줘청,왕이팅",00:47:00,20240115172619


In [164]:
date_data = datetime.datetime.strptime('20240115172619', '%Y%m%d%H%M%S').date()
date_data = datetime.datetime.strptime('20240115172619', '%Y%m%d%H%M%S').time()

print(date_data)

17:26:19


- 방영 시작 시간 - viewing_start_time

In [165]:
viewing_start_date = []
viewing_start_time = []

filtered_vod['viewing_start_time'] = filtered_vod['viewing_start_time'].astype(str)

for value in filtered_vod['viewing_start_time'] :
  date_data = datetime.datetime.strptime(value, '%Y%m%d%H%M%S').date()
  time_data = datetime.datetime.strptime(value, '%Y%m%d%H%M%S').time()
  viewing_start_date.append(date_data)
  viewing_start_time.append(time_data)

filtered_vod['viewing_start_date'] = viewing_start_date
filtered_vod['viewing_start_time'] = viewing_start_time

# viewing_start_time = pd.to_datetime(filtered_vod['viewing_start_time'], format= "%Y%m%d%H%M%S")
# filtered_vod['viewing_start_time'] = viewing_start_time

print(filtered_vod.head())


         ID         title category genre_category viewing_time  \
0  66875000   장군재상 10회(A)    TV드라마         외화 시리즈     00:41:12   
1  66705000  수호지의 무송 04회.    TV드라마         외화 시리즈     00:31:31   
2  66705000  수호지의 무송 04회.    TV드라마         외화 시리즈     00:10:32   
7  66875000    가남전 03회(A)    TV드라마         외화 시리즈     00:10:36   
8  66875000    가남전 03회(A)    TV드라마         외화 시리즈     00:29:49   

                                                SMRY                cast  \
0  조옥근은 엽소에게 호청의 짝사랑을 전하지만 엽소는 오히려 호청을 찾아가 호되게 나무...  마사순,성일륜,정천,왕초연,장준명   
1  무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...      유대경,번장강,손요기,장한   
2  무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...      유대경,번장강,손요기,장한   
7  황궁에서 다시 만난 이겸과 강보녕, 조소는 드디어 서로의 신분을 알게 되고, 이겸과...     쥐징이,쩡순시,왕줘청,왕이팅   
8  황궁에서 다시 만난 이겸과 강보녕, 조소는 드디어 서로의 신분을 알게 되고, 이겸과...     쥐징이,쩡순시,왕줘청,왕이팅   

  running_time viewing_start_time viewing_start_date  
0     00:43:00           08:51:02         2024-01-23  
1     00:45:00           20:37:41   

제목 끝에 . 제거

In [166]:
for idx in filtered_vod.index :
  filtered_vod.loc[idx, 'title'] = re.sub('[.]', '', filtered_vod.loc[idx, 'title'])

filtered_vod.head()

,ID,title,category,genre_category,viewing_time,SMRY,cast,running_time,viewing_start_time,viewing_start_date
0,66875000,장군재상 10회(A),TV드라마,외화 시리즈,00:41:12,조옥근은 엽소에게 호청의 짝사랑을 전하지만 엽소는 오히려 호청을 찾아가 호되게 나무...,"마사순,성일륜,정천,왕초연,장준명",00:43:00,08:51:02,2024-01-23
1,66705000,수호지의 무송 04회,TV드라마,외화 시리즈,00:31:31,무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...,"유대경,번장강,손요기,장한",00:45:00,20:37:41,2024-01-16
2,66705000,수호지의 무송 04회,TV드라마,외화 시리즈,00:10:32,무대는 무송을 집으로 데려와 회포를 푼다. 반금련은 훤칠하고 잘생긴 무송을 보고 못...,"유대경,번장강,손요기,장한",00:45:00,19:31:53,2024-01-17
7,66875000,가남전 03회(A),TV드라마,외화 시리즈,00:10:36,"황궁에서 다시 만난 이겸과 강보녕, 조소는 드디어 서로의 신분을 알게 되고, 이겸과...","쥐징이,쩡순시,왕줘청,왕이팅",00:47:00,23:48:23,2024-01-14
8,66875000,가남전 03회(A),TV드라마,외화 시리즈,00:29:49,"황궁에서 다시 만난 이겸과 강보녕, 조소는 드디어 서로의 신분을 알게 되고, 이겸과...","쥐징이,쩡순시,왕줘청,왕이팅",00:47:00,17:26:19,2024-01-15


성인, 키즈 제외 > 결측치 4개 삭제 > type : datetime > 제목 끝 . 제거

In [167]:
filtered_vod.to_csv(path_or_buf = './vod_전처리.csv', sep = ',', header= True,
                    mode= 'a', encoding= 'cp949')

movie / series 분리 > csv 저장

In [168]:
vod_movie = filtered_vod[filtered_vod['category'] == '영화']
vod_movie.info() #3852

vod_series = filtered_vod[filtered_vod['category'] != '영화']
vod_series.info() #13113

<class 'pandas.core.frame.DataFrame'>
Index: 3852 entries, 40 to 22553
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  3852 non-null   int64 
 1   title               3852 non-null   object
 2   category            3852 non-null   object
 3   genre_category      3852 non-null   object
 4   viewing_time        3852 non-null   object
 5   SMRY                3852 non-null   object
 6   cast                3852 non-null   object
 7   running_time        3852 non-null   object
 8   viewing_start_time  3852 non-null   object
 9   viewing_start_date  3852 non-null   object
dtypes: int64(1), object(9)
memory usage: 331.0+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 13113 entries, 0 to 22556
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  13113 non-null  int64 
 1   title          

In [169]:
vod_movie.to_csv(path_or_buf = './vod_movie_전처리.csv', sep = ',', header= True,
                    mode= 'a', encoding= 'cp949')

vod_series.to_csv(path_or_buf = './vod_series_전처리.csv', sep = ',', header= True,
                    mode= 'a', encoding= 'cp949')                    

In [1]:
###Efdsfsdfsdfsdf

In [2]:
print(1)

1
